In [1]:
import pandas as pd
import pymongo as pm
import numpy as np

## Extract

In [2]:
stroke_df = pd.read_csv('resources/healthcare-dataset-stroke-data.csv')
stroke_df.head()

,id,gender,age,hypertension,heart_disease,ever_married,work_type,Residence_type,avg_glucose_level,bmi,smoking_status,stroke
0,9046,Male,67.0,0,1,Yes,Private,Urban,228.69,36.6,formerly smoked,1
1,51676,Female,61.0,0,0,Yes,Self-employed,Rural,202.21,NaN,never smoked,1
2,31112,Male,80.0,0,1,Yes,Private,Rural,105.92,32.5,never smoked,1
3,60182,Female,49.0,0,0,Yes,Private,Urban,171.23,34.4,smokes,1
4,1665,Female,79.0,1,0,Yes,Self-employed,Rural,174.12,24.0,never smoked,1


In [3]:
framingham_df = pd.read_csv('resources/framingham.csv')
framingham_df.head()

,male,age,education,currentSmoker,cigsPerDay,BPMeds,prevalentStroke,prevalentHyp,diabetes,totChol,sysBP,diaBP,BMI,heartRate,glucose,TenYearCHD
0,1,39,4.0,0,0.0,0.0,0,0,0,195.0,106.0,70.0,26.97,80.0,77.0,0
1,0,46,2.0,0,0.0,0.0,0,0,0,250.0,121.0,81.0,28.73,95.0,76.0,0
2,1,48,1.0,1,20.0,0.0,0,0,0,245.0,127.5,80.0,25.34,75.0,70.0,0
3,0,61,3.0,1,30.0,0.0,0,1,0,225.0,150.0,95.0,28.58,65.0,103.0,1
4,0,46,3.0,1,23.0,0.0,0,0,0,285.0,130.0,84.0,23.10,85.0,85.0,0


## Clean

In [4]:
# Remove columns that are not in common with framingham
drop_columns = [
    'id', 
    'ever_married', 
    'work_type', 
    'Residence_type',
    'heart_disease'
]
stroke_df.drop(columns=drop_columns, inplace=True)
stroke_df.head()

,gender,age,hypertension,avg_glucose_level,bmi,smoking_status,stroke
0,Male,67.0,0,228.69,36.6,formerly smoked,1
1,Female,61.0,0,202.21,NaN,never smoked,1
2,Male,80.0,0,105.92,32.5,never smoked,1
3,Female,49.0,0,171.23,34.4,smokes,1
4,Female,79.0,1,174.12,24.0,never smoked,1


In [5]:
# Remove columns not in common with stroke data
drop_columns = [
    'cigsPerDay', 
    'BPMeds', 
    'totChol', 
    'sysBP', 
    'diaBP', 
    'heartRate', 
    'TenYearCHD',
    'education',
    'diabetes'
]
framingham_df.drop(columns=drop_columns, inplace=True)
framingham_df.head()

,male,age,currentSmoker,prevalentStroke,prevalentHyp,BMI,glucose
0,1,39,0,0,0,26.97,77.0
1,0,46,0,0,0,28.73,76.0
2,1,48,1,0,0,25.34,70.0
3,0,61,1,0,1,28.58,103.0
4,0,46,1,0,0,23.10,85.0


In [6]:
# Make the gender match stroke data.
framingham_df['gender'] = np.where(framingham_df['male'] == 1, 'Male', 'Female')
framingham_df.drop(columns='male', inplace=True)
framingham_df.head()

,age,currentSmoker,prevalentStroke,prevalentHyp,BMI,glucose,gender
0,39,0,0,0,26.97,77.0,Male
1,46,0,0,0,28.73,76.0,Female
2,48,1,0,0,25.34,70.0,Male
3,61,1,0,1,28.58,103.0,Female
4,46,1,0,0,23.10,85.0,Female


In [7]:
# Shape the smoker to match framingham
stroke_df['current_smoker'] = np.where(stroke_df['smoking_status'] == 'smokes', 1, 0)
stroke_df.drop(columns=['smoking_status'], inplace=True)
stroke_df.head()

,gender,age,hypertension,avg_glucose_level,bmi,stroke,current_smoker
0,Male,67.0,0,228.69,36.6,1,0
1,Female,61.0,0,202.21,NaN,1,0
2,Male,80.0,0,105.92,32.5,1,0
3,Female,49.0,0,171.23,34.4,1,1
4,Female,79.0,1,174.12,24.0,1,0


In [8]:
# standarize the columns
rename_columns = {
    'currentSmoker': 'current_smoker',
    'BMI': 'bmi',
    'glucose': 'avg_glucose_level',
    'prevalentStroke': 'stroke',
    'prevalentHyp': 'hypertension'
}
framingham_df.rename(columns=rename_columns, inplace=True)
framingham_df.head()

,age,current_smoker,stroke,hypertension,bmi,avg_glucose_level,gender
0,39,0,0,0,26.97,77.0,Male
1,46,0,0,0,28.73,76.0,Female
2,48,1,0,0,25.34,70.0,Male
3,61,1,0,1,28.58,103.0,Female
4,46,1,0,0,23.10,85.0,Female


In [9]:
# Format datatypes to match.
framingham_df['age'] =  pd.to_numeric(framingham_df['age'], downcast='float')
framingham_df.head()

,age,current_smoker,stroke,hypertension,bmi,avg_glucose_level,gender
0,39.0,0,0,0,26.97,77.0,Male
1,46.0,0,0,0,28.73,76.0,Female
2,48.0,1,0,0,25.34,70.0,Male
3,61.0,1,0,1,28.58,103.0,Female
4,46.0,1,0,0,23.10,85.0,Female


In [10]:
stroke_df.size

35770

In [11]:
# Merge the data
stroke_df = pd.concat([framingham_df, stroke_df])
stroke_df.size

65436

In [12]:
stroke_df.head()

,age,current_smoker,stroke,hypertension,bmi,avg_glucose_level,gender
0,39.0,0,0,0,26.97,77.0,Male
1,46.0,0,0,0,28.73,76.0,Female
2,48.0,1,0,0,25.34,70.0,Male
3,61.0,1,0,1,28.58,103.0,Female
4,46.0,1,0,0,23.10,85.0,Female


In [13]:
# Remove and NA
stroke_df.dropna(inplace=True)
stroke_df.size 

61215

In [14]:
# Remove the children
stroke_df = stroke_df[stroke_df['age'] >= 18.0]
stroke_df.size

55363

## Save

In [15]:
# Save to CSV for tableau
stroke_df.to_csv('resources/clean_stroke_data.csv')